In [1]:
import pandas as pd

df = pd.read_csv('/content/Mental-health-related-subreddits.csv', low_memory=False)
display(df.head())

,Title,Text,Subreddit
0,exposure does not work!,I have struggled with social anxiety from chil...,Anxiety
1,Panic attack? derealization? can't go to docto...,"Back in March (I know, a while ago D:), I woke...",Anxiety
2,How long can a panic attack last?!,I've been withdrawing from medicines lately (e...,Anxiety
3,Stepping stones,"First time poster, long time lurker. \n\nI've ...",Anxiety
4,"Coping with anxiety over climate change, on th...","Hi all,\n\nI made a throwaway account as my ma...",Anxiety


In [2]:
import sys
!{sys.executable} -m pip install nltk --quiet
!{sys.executable} -m pip install wordcloud --quiet
import zipfile
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import re
from wordcloud import WordCloud
from collections import Counter
from nltk.corpus import stopwords

In [3]:
print(df.columns)

Index(['Title', 'Text', 'Subreddit'], dtype='object')


In [4]:
print("SHAPE BEFORE CLEANING: ", df.shape)

SHAPE BEFORE CLEANING:  (488472, 3)


In [5]:
df = df.loc[:, ~df.columns.str.startswith('Unnamed')]
print(df.head())

                                               Title  \
0                            exposure does not work!   
1  Panic attack? derealization? can't go to docto...   
2                 How long can a panic attack last?!   
3                                    Stepping stones   
4  Coping with anxiety over climate change, on th...   

                                                Text Subreddit  
0  I have struggled with social anxiety from chil...   Anxiety  
1  Back in March (I know, a while ago D:), I woke...   Anxiety  
2  I've been withdrawing from medicines lately (e...   Anxiety  
3  First time poster, long time lurker. \n\nI've ...   Anxiety  
4  Hi all,\n\nI made a throwaway account as my ma...   Anxiety  


In [6]:
print(df.columns)

Index(['Title', 'Text', 'Subreddit'], dtype='object')


In [7]:
print("SHAPE AFTER CLEANING: ", df.shape)

SHAPE AFTER CLEANING:  (488472, 3)


In [8]:
df['Combined'] = df['Title'].fillna('') + ' ' + df['Text'].fillna('')

In [9]:
df['Combined'] = df['Combined'].str.lower()
df['Combined'] = df['Combined'].str.replace(r'[^a-z\s]', '', regex=True)
df.head

<bound method NDFrame.head of                                                     Title  \
0                                 exposure does not work!   
1       Panic attack? derealization? can't go to docto...   
2                      How long can a panic attack last?!   
3                                         Stepping stones   
4       Coping with anxiety over climate change, on th...   
...                                                   ...   
488467                 Religion and paranoid conversation   
488468  Isn't this a little weird for being tardive dy...   
488469  Is this a symptom of schizophrenia or somethin...   
488470  Anyone seen the new Black Mirror episode Bande...   
488471   My girlfriend of over 1 year and I just broke up   

                                                     Text      Subreddit  \
0       I have struggled with social anxiety from chil...        Anxiety   
1       Back in March (I know, a while ago D:), I woke...        Anxiety   
2       I've been withdrawing from medicines lately (e...        Anxiety   
3       First time poster, long time lurker. \n\nI've ...        Anxiety   
4       Hi all,\n\nI made a throwaway account as my ma...        Anxiety   
...                                                   ...            ...   
488467  Ok! I've got a rant to make and a conversation...  schizophrenia   
488468  When I wake up, I rarely have any tics in my t...  schizophrenia   
488469  I’m not sure if this is schizophrenia (or perh...  schizophrenia   
488470  Although I don't think it was stated what Stef...  schizophrenia   
488471  It's really tough right now. We broke up due t...  schizophrenia   

                                                 Combined  
0       exposure does not work i have struggled with s...  
1       panic attack derealization cant go to doctors ...  
2       how long can a panic attack last ive been with...  
3       stepping stones first time poster long time lu...  
4       coping with anxiety over climate change on the...  
...                                                   ...  
488467  religion and paranoid conversation ok ive got ...  
488468  isnt this a little weird for being tardive dys...  
488469  is this a symptom of schizophrenia or somethin...  
488470  anyone seen the new black mirror episode bande...  
488471  my girlfriend of over  year and i just broke u...  

[488472 rows x 4 columns]>

In [10]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

df['Combined'] = df['Combined'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('omw-1.4')

lemmatizer = WordNetLemmatizer()

df['Combined'] = df['Combined'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in x.split()]))

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [12]:
print("Lemmatized Text Samples: \n")
for i in range(5):
    print(f"Post #{i + 1}: ")
    print(df['Combined'].iloc[i])
    print("-" * 80)

Lemmatized Text Samples: 

Post #1: 
exposure work struggled social anxiety childhood main advice friend selfhelp book professional expose environment im better example get anxious every single time go shop even though shopping daily weekly since get nervous sweaty heart pounding every time go gym yet working gym year exposure worked anyone else
--------------------------------------------------------------------------------
Post #2: 
panic attack derealization cant go doctor advice appreciated back march know ago woke day go school feeling kind felt depressed didnt really feel like getting even seeing anyone history p really mainly happens winter time get highly anxious also going anywhere period time get suspicious paranoid everyone anything usually end isolating room eat way less also get highly irritable emotional like one time mom sent store milk started cry store thought going killed watched believe period watched government something higher power im gerbil study something like t

In [13]:
df = df[df['Combined'].str.len() > 10]
df.isnull().values.any()
df.isnull().sum().sum()
df.isnull().sum()

,0
Title,5
Text,0
Subreddit,3
Combined,0


In [14]:
df[df.isnull().any(axis=1)]

,Title,Text,Subreddit,Combined
228051,NaN,Anyone have an answer to this? I've been askin...,NaN,anyone answer ive asking cant figure
229087,NaN,I'm slowly being worn down to nothing. I used ...,depression,im slowly worn nothing used able take step bac...
437117,NaN,Not too much available online regarding this. ...,NaN,much available online regarding anyone ittried
441637,NaN,Heat wave and haven't slept more than 4ish hou...,NaN,heat wave havent slept ish hour night hot fan ...
449208,NaN,Sitting here in this pub drinking a beer. The ...,bipolar,sitting pub drinking beer old woman screaming ...


In [15]:
df.dropna(subset=['Title', 'Text', 'Subreddit', 'Combined'], inplace=True)

/tmp/ipython-input-724578079.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(subset=['Title', 'Text', 'Subreddit', 'Combined'], inplace=True)


In [16]:
df.reset_index(drop=True, inplace=True)

In [17]:
df['Text'] = df['Text'].str.replace('\n', ' ', regex=True)
df['Title'] = df['Title'].str.replace('\n', ' ', regex=True)
df['Combined'] = df['Combined'].str.replace('\n', ' ', regex=True)

/tmp/ipython-input-3396328298.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Text'] = df['Text'].str.replace('\n', ' ', regex=True)
/tmp/ipython-input-3396328298.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Title'] = df['Title'].str.replace('\n', ' ', regex=True)
/tmp/ipython-input-3396328298.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https

In [18]:
df.isnull().values.any()
df.isnull().sum().sum()
df.isnull().sum()

,0
Title,0
Text,0
Subreddit,0
Combined,0


In [19]:
from IPython.display import display
display(df)

,Title,Text,Subreddit,Combined
0,exposure does not work!,I have struggled with social anxiety from chil...,Anxiety,exposure work struggled social anxiety childho...
1,Panic attack? derealization? can't go to docto...,"Back in March (I know, a while ago D:), I woke...",Anxiety,panic attack derealization cant go doctor advi...
2,How long can a panic attack last?!,I've been withdrawing from medicines lately (e...,Anxiety,long panic attack last ive withdrawing medicin...
3,Stepping stones,"First time poster, long time lurker. I've ha...",Anxiety,stepping stone first time poster long time lur...
4,"Coping with anxiety over climate change, on th...","Hi all, I made a throwaway account as my main...",Anxiety,coping anxiety climate change verge considerin...
...,...,...,...,...
488300,Religion and paranoid conversation,Ok! I've got a rant to make and a conversation...,schizophrenia,religion paranoid conversation ok ive got rant...
488301,Isn't this a little weird for being tardive dy...,"When I wake up, I rarely have any tics in my t...",schizophrenia,isnt little weird tardive dyskinesia wake rare...
488302,Is this a symptom of schizophrenia or somethin...,I’m not sure if this is schizophrenia (or perh...,schizophrenia,symptom schizophrenia something else im sure s...
488303,Anyone seen the new Black Mirror episode Bande...,Although I don't think it was stated what Stef...,schizophrenia,anyone seen new black mirror episode bandersna...


In [20]:
# Define the valid subreddits you want to keep
valid_subreddits = ['depression', 'Anxiety', 'bipolar', 'mentalhealth', 'BPD', 'schizophrenia', 'autism']

# Replace all other subreddit values with NaN
df['Subreddit'] = df['Subreddit'].where(df['Subreddit'].isin(valid_subreddits))

# Drop rows where 'subreddit' is NaN (i.e., invalid or unwanted subreddits)
df.dropna(subset=['Subreddit'], inplace=True)
df.isnull().values.any()

/tmp/ipython-input-901992291.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Subreddit'] = df['Subreddit'].where(df['Subreddit'].isin(valid_subreddits))
/tmp/ipython-input-901992291.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(subset=['Subreddit'], inplace=True)


False

In [21]:
df.reset_index(drop=True, inplace=True)
print("SHAPE AFTER CLEANING: ", df.shape)

SHAPE AFTER CLEANING:  (488305, 4)


In [22]:
from sklearn.feature_extraction.text import CountVectorizer
from collections import defaultdict

In [23]:
def simple_clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [24]:
import numpy as np

top_words_per_class = defaultdict(list)
vectorizer = CountVectorizer(max_features=1000)
for sub in df['Subreddit'].unique():
    texts = df[df['Subreddit'] == sub]['Combined']
    X = vectorizer.fit_transform(texts)
    word_freq = np.asarray(X.sum(axis=0)).flatten()
    vocab = vectorizer.get_feature_names_out()
    freq_dict = dict(zip(vocab, word_freq))
    sorted_words = sorted(freq_dict.items(), key=lambda x: x[1], reverse=True)[:10]
    top_words_per_class[sub] = sorted_words

In [25]:
top_words_per_class_df = pd.DataFrame({k: dict(v) for k, v in top_words_per_class.items()})
display(top_words_per_class_df)

,Anxiety,BPD,depression,autism,mentalhealth,bipolar,schizophrenia
im,176004.0,77812.0,572390.0,8736.0,91399.0,76926.0,25541.0
anxiety,134973.0,NaN,NaN,NaN,NaN,NaN,NaN
like,108925.0,55472.0,357957.0,6944.0,64665.0,42289.0,17854.0
feel,95585.0,48513.0,347448.0,NaN,55273.0,34981.0,11136.0
get,76384.0,28762.0,215686.0,4726.0,39487.0,26876.0,9684.0
dont,74713.0,40023.0,348214.0,4837.0,53529.0,30735.0,12485.0
ive,70124.0,27657.0,208422.0,NaN,37206.0,30052.0,8928.0
time,65537.0,29065.0,204083.0,3930.0,38455.0,26486.0,9606.0
know,65372.0,35830.0,236676.0,4814.0,43141.0,26558.0,11033.0
really,54596.0,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

vectorizer = TfidfVectorizer(max_features=5000)

In [27]:
# Drop classes with fewer than 2 samples
min_samples = 2
class_counts = df['Subreddit'].value_counts()
valid_classes = class_counts[class_counts >= min_samples].index
df = df[df['Subreddit'].isin(valid_classes)]

# Recreate features after filtering
X = vectorizer.fit_transform(df['Combined'])
y = df['Subreddit']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
print(y.value_counts())

Subreddit
depression       258396
Anxiety           86229
bipolar           41480
mentalhealth      39369
BPD               38197
schizophrenia     17493
autism             7141
Name: count, dtype: int64


In [28]:
print("Train Set Class Distribution:\n", y_train.value_counts(normalize=True))
print("\nTest Set Class Distribution:\n", y_test.value_counts(normalize=True))

Train Set Class Distribution:
 Subreddit
depression       0.529170
Anxiety          0.176588
bipolar          0.084947
mentalhealth     0.080623
BPD              0.078225
schizophrenia    0.035823
autism           0.014625
Name: proportion, dtype: float64

Test Set Class Distribution:
 Subreddit
depression       0.529167
Anxiety          0.176590
bipolar          0.084947
mentalhealth     0.080626
BPD              0.078220
schizophrenia    0.035828
autism           0.014622
Name: proportion, dtype: float64


In [29]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('punkt_tab')  # less common, but required in some versions
nltk.download("stopwords")

stop_words = set(stopwords.words("english"))

def clean_and_tokenize(text):
    text = re.sub(r"http\S+|[^a-zA-Z\s]", "", text.lower())  # clean URLs & punctuation
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words and len(word) > 2]
    return tokens

df["Tokens"] = df["Combined"].apply(clean_and_tokenize)

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [30]:
!pip install gensim


In [31]:
#Creating dictionaries
from gensim import corpora
dictionary = corpora.Dictionary(df["Tokens"])
# Create bag-of-words corpus
corpus = [dictionary.doc2bow(text) for text in df["Tokens"]]

In [32]:
#Training the LDA Model

from gensim.models.ldamodel import LdaModel

lda_model = LdaModel(
    corpus=corpus,
    id2word=dictionary,
    num_topics=7,       # Try 5–10 to start
    random_state=42,
    passes=10
)

In [33]:
#Viewing the topics
topics = lda_model.print_topics(num_words=10)
for topic in topics:
    print(topic)

(0, '0.020*"job" + 0.020*"work" + 0.016*"get" + 0.014*"year" + 0.012*"time" + 0.011*"school" + 0.011*"dont" + 0.010*"ive" + 0.010*"life" + 0.009*"going"')
(1, '0.012*"room" + 0.010*"look" + 0.010*"music" + 0.009*"game" + 0.008*"food" + 0.008*"fuck" + 0.008*"eat" + 0.008*"voice" + 0.007*"fucking" + 0.007*"video"')
(2, '0.039*"dont" + 0.035*"like" + 0.032*"feel" + 0.025*"know" + 0.023*"want" + 0.014*"cant" + 0.014*"really" + 0.014*"people" + 0.014*"get" + 0.012*"think"')
(3, '0.017*"people" + 0.009*"thing" + 0.009*"schizophrenia" + 0.008*"life" + 0.008*"would" + 0.007*"one" + 0.006*"like" + 0.006*"ampxb" + 0.006*"thought" + 0.006*"think"')
(4, '0.012*"friend" + 0.011*"year" + 0.011*"time" + 0.011*"didnt" + 0.010*"would" + 0.010*"got" + 0.010*"said" + 0.010*"told" + 0.009*"one" + 0.007*"back"')
(5, '0.017*"ive" + 0.015*"bipolar" + 0.013*"med" + 0.011*"help" + 0.010*"medication" + 0.009*"year" + 0.009*"anyone" + 0.009*"doctor" + 0.008*"know" + 0.008*"diagnosed"')
(6, '0.027*"like" + 0.024*

In [34]:
(0, '0.05*"panic" + 0.04*"attack" + 0.03*"anxiety" + ...')

(0, '0.05*"panic" + 0.04*"attack" + 0.03*"anxiety" + ...')

In [35]:
#Assigning the topics
df["Topic_Distribution"] = [lda_model.get_document_topics(doc) for doc in corpus]


In [36]:
df["Dominant_Topic"] = df["Topic_Distribution"].apply(lambda x: sorted(x, key=lambda x: -x[1])[0][0])


In [37]:
#Visualization of LDA
!pip install pyLDAvis


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 39.6 MB/s eta 0:00:00


In [38]:
!pip install pyLDAvis

import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

pyLDAvis.enable_notebook()

vis = gensimvis.prepare(lda_model, corpus, dictionary)
vis


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.082119 -0.017230       1        1  30.541275
4      0.055506 -0.022990       2        1  18.882193
0      0.071728 -0.068039       3        1  13.660614
3      0.055599  0.234540       4        1  12.419332
5      0.135596  0.017477       5        1  11.493409
6      0.003915 -0.161574       6        1   9.666940
1     -0.404463  0.017816       7        1   3.336237, topic_info=        Term           Freq          Total Category  logprob  loglift
77      dont  636932.000000  636932.000000  Default  30.0000  30.0000
92      feel  589399.000000  589399.000000  Default  29.0000  29.0000
730      job  122239.000000  122239.000000  Default  28.0000  28.0000
154   people  302228.000000  302228.000000  Default  27.0000  27.0000
130     like  746689.000000  746689.000000  Default  26.0000  26.0000
...      ...            ...            ...      ...      ...      ...
2375    hear    8853.027322   50831.841347   Topic7  -5.1269   1.6526
473      see    8689.955058  144952.589418   Topic7  -5.1455   0.5861
193     take    7864.838099  122710.641895   Topic7  -5.2452   0.6529
160      put    7067.956567   53843.203424   Topic7  -5.3521   1.3698
16       get    7799.245085  417917.408323   Topic7  -5.2536  -0.5809

[569 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
1955      5  0.999889  abilify
1216      2  0.956659  abusive
1216      4  0.043162  abusive
1445      6  0.999354     ache
2793      7  0.998867     acne
...     ...       ...      ...
37        4  0.001780     year
37        5  0.190233     year
37        6  0.005394     year
1089      5  0.999728   zoloft
262       7  0.999836  zyprexa

[1262 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 5, 1, 4, 6, 7, 2])

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag